# Homework Week 3

## 1 & 2 :Prefect & Version

In [2]:
!prefect version

Version:             3.2.14
API version:         0.8.4
Python version:      3.12.9
Git commit:          efcde6dc
Built:               Fri, Mar 21, 2025 5:28 PM
OS/Arch:             darwin/arm64
Profile:             local
Server type:         ephemeral
Pydantic version:    2.10.6
Server:
  Database:          sqlite
  SQLite version:    3.43.2


## 3. Size of Df

In [5]:
import pandas as pd

In [6]:
file_path = 'week3/data/yellow_tripdata_2023-03.parquet'

In [7]:
df = pd.read_parquet(file_path)

In [9]:
df.shape

(3403766, 19)

## 4. Data Preprocessing

In [10]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [11]:
df = read_dataframe(file_path)

In [12]:
df.shape

(3316216, 20)

## 5. Pipeline

Used prefect to create a workflow

In [13]:
!cat week3/duration_prediction.py

#!/usr/bin/env python
# coding: utf-8

import pickle
from pathlib import Path

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from prefect import flow, task
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("nyc-taxi-experiment")

models_folder = Path('models')
models_folder.mkdir(exist_ok=True)


@task
def read_dataframe(year, month):
    url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet'
    print ("reading from :: ", url)
    return read_dataframe_with_path(url)


def read_dataframe_with_path(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    